<a href="https://colab.research.google.com/github/HungTran1294/ASR-FOR-VN-Challenge-SOICT/blob/main/Fine_tuning_Wav2Vec2_for_VN_for_ASR_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Đọc hướng dẫn này mới chạy:
1.  Các chỗ cần sửa:
  - data_dir cập nhật path drive đến thư mục chưa dataset
  - reponame tên của repo sẽ là tên repo trên huggingface và thư mục lưu trên driver
  - output_dir path lưu check point trên driver
2. chuẩn bị token:
  - wandb
  - hugging face
  -



In [1]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
# Giữ nguyên các import khác từ notebook của bạn
import os
import re
import random
import pandas as pd
from IPython.display import display, HTML
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer
from huggingface_hub import notebook_login

In [2]:
from google.colab import drive
#Đường dẫn để lưu và tải dataset trên Google Drive
processed_dataset_path = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k"

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import os
from datasets import DatasetDict

# Thay thế bằng đường dẫn thực tế của bạn
data_dir = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02"

# Kiểm tra xem dataset đã tồn tại trên Drive chưa
if os.path.exists(os.path.join(processed_dataset_path, "train")) and \
   os.path.exists(os.path.join(processed_dataset_path, "test")):
    print(f"Dataset đã tồn tại tại {processed_dataset_path}. Đang tải từ Drive...")
    # Tải dataset train và test
    loaded_train_dataset = load_from_disk(os.path.join(processed_dataset_path, "train"))
    loaded_test_dataset = load_from_disk(os.path.join(processed_dataset_path, "test"))

    # Gán lại cho biến timit để sử dụng
    timit = DatasetDict({
        "train": loaded_train_dataset,
        "test": loaded_test_dataset
    })
    print("Dataset đã được tải thành công từ Drive:")
    print(timit)

else:
    print(f"Dataset chưa tồn tại tại {processed_dataset_path}. Đang tiến hành tiền xử lý...")
    # --- Bắt đầu phần code tiền xử lý dataset của bạn ---

    # Thay thế bằng đường dẫn thực tế của bạn đến thư mục chứa dữ liệu gốc

    audio_files_valid = []
    transcript_files_valid = []

    # Lấy danh sách tất cả các tệp .wav và .txt trong thư mục và thư mục con
    all_files = {}
    for root, _, files in os.walk(data_dir):
        for file in files:
            name, ext = os.path.splitext(file)
            if ext in ['.wav', '.txt']:
                if name not in all_files:
                    all_files[name] = {}
                all_files[name][ext] = os.path.join(root, file)

    # Kiểm tra các cặp audio và transcript hợp lệ
    for name, files_dict in all_files.items():
        if '.wav' in files_dict and '.txt' in files_dict:
            audio_files_valid.append(files_dict['.wav'])
            transcript_files_valid.append(files_dict['.txt'])
        else:
            print(f"Bỏ qua cặp không khớp: {name}") # In thông báo nếu có cặp không khớp

    data = {"audio_file": audio_files_valid, "transcript_file": transcript_files_valid}
    dataset = Dataset.from_dict(data)

    # Bây giờ bạn có thể thêm bước load nội dung transcript vào dataset
    def load_transcript(batch):
        with open(batch["transcript_file"], "r", encoding="utf-8") as f:
            batch["text"] = f.read().strip()
        return batch

    dataset = dataset.map(load_transcript)

    # Bạn cũng cần load dữ liệu âm thanh
    def load_audio(batch):
        # Bạn có thể sử dụng librosa hoặc torchaudio để load audio
        import librosa
        audio, sampling_rate = librosa.load(batch["audio_file"], sr=16000) # Thay 16000 bằng sampling rate mong muốn
        batch["audio"] = {"array": audio, "sampling_rate": sampling_rate}
        return batch

    dataset = dataset.map(load_audio)

    # Xử lý ký tự đặc biệt
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    def remove_special_characters(batch):
        batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
        return batch

    dataset = dataset.map(remove_special_characters)

    print("Dataset ban đầu sau khi xử lý ký tự đặc biệt và bỏ qua các cặp không khớp:")
    print(dataset)

    # Tách dataset thành train và test với tỷ lệ 90/10
    train_test_dataset = dataset.train_test_split(test_size=0.1)

    # Gán lại tên cho dễ sử dụng
    timit = train_test_dataset

    print("Dataset sau khi tách:")
    print(timit)

    # --- Kết thúc phần code tiền xử lý dataset của bạn ---

    # Tạo thư mục lưu nếu chưa có
    os.makedirs(processed_dataset_path, exist_ok=True)

    # Lưu dataset train và test sau khi tiền xử lý xong
    timit["train"].save_to_disk(os.path.join(processed_dataset_path, "train"))
    timit["test"].save_to_disk(os.path.join(processed_dataset_path, "test"))

    print(f"Dataset đã được tiền xử lý và lưu tại: {processed_dataset_path}")


Dataset chưa tồn tại tại /content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k. Đang tiến hành tiền xử lý...
Bỏ qua cặp không khớp: speaker_655-099991-1
Bỏ qua cặp không khớp: speaker_041-125378-1
Bỏ qua cặp không khớp: database_sa3_Jan21_Apr24_cleaned_utt_0000082117-1
Bỏ qua cặp không khớp: database_sa3_Jan21_Apr24_cleaned_utt_0000097267-1
Bỏ qua cặp không khớp: database_sa3_Jan21_Apr24_cleaned_utt_0000114482-1
Bỏ qua cặp không khớp: database_sa4_Mar17_toApril11_cleaned_utt_0000095295-1
Bỏ qua cặp không khớp: speaker_485-095271-1
Bỏ qua cặp không khớp: database_sa1_Mar20_May04_cleaned_utt_0000038292-1
Bỏ qua cặp không khớp: spkyut-20190730-utt000003094
Bỏ qua cặp không khớp: database_sa2_Jan4_Feb29_cleaned_utt_0000083263-1
Bỏ qua cặp không khớp: database_sa3_Jan21_Apr24_cleaned_utt_0000026263-1
Bỏ qua cặp không khớp: speaker_774-099571-1
Bỏ qua cặp không khớp: spkyut-20190730-utt000004923
Bỏ qua cặp không khớp: database_sa3_1_150h_15Jan202

Map:   0%|          | 0/20204 [00:00<?, ? examples/s]

Map:   0%|          | 0/20204 [00:00<?, ? examples/s]

Map:   0%|          | 0/20204 [00:00<?, ? examples/s]

Dataset ban đầu sau khi xử lý ký tự đặc biệt và bỏ qua các cặp không khớp:
Dataset({
    features: ['audio_file', 'transcript_file', 'text', 'audio'],
    num_rows: 20204
})
Dataset sau khi tách:
DatasetDict({
    train: Dataset({
        features: ['audio_file', 'transcript_file', 'text', 'audio'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['audio_file', 'transcript_file', 'text', 'audio'],
        num_rows: 2021
    })
})


Saving the dataset (0/16 shards):   0%|          | 0/18183 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/2021 [00:00<?, ? examples/s]

Dataset đã được tiền xử lý và lưu tại: /content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k


# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, we will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on any English ASR dataset. Note that in this notebook, we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, we fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [Timit](https://huggingface.co/datasets/timit_asr) dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition.

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, one has a much easier time getting access to a V100 or P100 GPU however.

In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 18 05:31:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [11]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training.

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [12]:
%%capture
!apt install git-lfs




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

In [13]:
timit

DatasetDict({
    train: Dataset({
        features: ['audio_file', 'transcript_file', 'text', 'audio'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['audio_file', 'transcript_file', 'text', 'audio'],
        num_rows: 2021
    })
})

Many ASR datasets only provide the target text, `'text'` for each audio `'audio'` and file `'file'`. Timit actually provides much more information about each audio file, such as the `'phonetic_detail'`, etc., which is why many researchers choose to evaluate their models on phoneme classification instead of speech recognition when working with Timit. However, we want to keep the notebook as general as possible, so that we will only consider the transcribed text for fine-tuning.



In [ ]:
# timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [14]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(timit["train"], num_examples=10)

,audio_file,transcript_file,text,audio
0,/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02/database_sa1_Mar20_May04_cleaned_utt_0000067208-1.wav,/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02/database_sa1_Mar20_May04_cleaned_utt_0000067208-1.txt,và ngay bây giờ chúng ta sẽ cùng nhau đến với vòng xoay may mắn các bạn sẽ có cơ hội nhận về cho mình là một com bồ váy cưới cao cấp trị giá mười lăm triệu đồng,"{'array': [-0.021148681640625, -0.05621337890625, -0.091400146484375, -0.061309814453125, -0.062835693359375, -0.03778076171875, -0.0386962890625, -0.063873291015625, -0.027923583984375, -0.0147705078125, 0.02239990234375, 0.0762939453125, 0.05340576171875, 0.06231689453125, 0.0853271484375, 0.064361572265625, 0.0860595703125, 0.099273681640625, 0.115203857421875, 0.1578369140625, 0.134674072265625, 0.137176513671875, 0.1566162109375, 0.156402587890625, 0.19927978515625, 0.20819091796875, 0.2164306640625, 0.23260498046875, 0.20880126953125, 0.223663330078125, 0.214263916015625, 0.166748046875, 0.184295654296875, 0.19268798828125, 0.16961669921875, 0.19000244140625, 0.1943359375, 0.224822998046875, 0.2252197265625, 0.1556396484375, 0.148651123046875, 0.142120361328125, 0.114013671875, 0.143646240234375, 0.1417236328125, 0.118072509765625, 0.1087646484375, 0.0821533203125, 0.10137939453125, 0.092193603515625, 0.0411376953125, 0.02410888671875, -0.01190185546875, -0.015228271484375, -0.008941650390625, -0.033660888671875, -0.019439697265625, -0.00640869140625, -0.0205078125, -0.0008544921875, -0.035125732421875, -0.077301025390625, -0.05645751953125, -0.057952880859375, -0.01556396484375, 0.03228759765625, 0.049285888671875, 0.077392578125, 0.05377197265625, 0.0166015625, 0.0318603515625, 0.038665771484375, 0.05865478515625, 0.073883056640625, 0.052825927734375, 0.06072998046875, 0.063568115234375, 0.075714111328125, 0.122100830078125, 0.121368408203125, 0.12042236328125, 0.134185791015625, 0.10772705078125, 0.11083984375, 0.119659423828125, 0.087127685546875, 0.091705322265625, 0.05401611328125, 0.012542724609375, 0.006256103515625, -0.0469970703125, -0.06097412109375, -0.06884765625, -0.09979248046875, -0.08544921875, -0.06195068359375, -0.07427978515625, -0.077392578125, -0.101226806640625, -0.116607666015625, ...], 'sampling_rate': 16000}"
1,/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02/database_sa2_Jan4_Feb29_cleaned_utt_0000128033-1.wav,/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02/database_sa2_Jan4_Feb29_cleaned_utt_0000128033-1.txt,đây là bất ngờ của lễ trao giải vì bộ phim này vốn chỉ được đánh giá là ứng cử viên thứ hai cho giải thưởng lớn nhưng lại giành được vị trí cao nhất liên hoan phim năm nay danh hiệu nữ diễn viên xuất sắc nhất,"{'array': [-0.005340576171875, -0.0107421875, -0.01287841796875, -0.01226806640625, -0.002655029296875, 0.014739990234375, 0.01593017578125, 0.00982666015625, 0.000457763671875, -0.0118408203125, -0.02374267578125, -0.023590087890625, -0.012939453125, 0.00579833984375, 0.019866943359375, 0.01861572265625, 0.01177978515625, 0.003875732421875, -0.002227783203125, -0.013946533203125, -0.01776123046875, -0.01995849609375, -0.00762939453125, 0.0067138671875, 0.020355224609375, 0.016204833984375, 0.002166748046875, -0.011077880859375, -0.022430419921875, -0.023468017578125, -0.016082763671875, 0.00201416015625, 0.01690673828125, 0.02679443359375, 0.02239990234375, 0.017578125, 0.0093994140625, -0.003204345703125, -0.012054443359375, -0.0118408203125, -0.009307861328125, 0.00054931640625, 0.005584716796875, -0.00274658203125, -0.01007080078125, -0.01080322265625, -0.017608642578125, -0.016998291015625, -0.002838134765625, 0.003265380859375, 0.00390625, 0.009613037109375, 0.0126953125, 0.009063720703125

Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [Timit](https://huggingface.co/datasets/timit_asr) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
# import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

# def remove_special_characters(batch):
#     batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
#     return batch

In [ ]:
# timit = timit.map(remove_special_characters)

In [ ]:
# show_random_elements(timit["train"])

Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [19]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

Map:   0%|          | 0/18183 [00:00<?, ? examples/s]

Map:   0%|          | 0/2021 [00:00<?, ? examples/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [20]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [30]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ê': 0,
 'ẵ': 1,
 'n': 2,
 'ế': 3,
 'ả': 4,
 'y': 5,
 'ụ': 6,
 'ẽ': 7,
 'z': 8,
 'e': 9,
 'ẹ': 10,
 'u': 11,
 'ý': 12,
 'm': 13,
 'i': 14,
 'ẻ': 15,
 'ỗ': 16,
 'ấ': 17,
 'ẩ': 18,
 'a': 19,
 'ừ': 20,
 '<': 21,
 'ỵ': 22,
 'd': 23,
 'ễ': 24,
 'ặ': 25,
 '>': 26,
 'ộ': 27,
 'ó': 28,
 'ở': 29,
 'ữ': 30,
 'ớ': 31,
 'h': 32,
 'ợ': 33,
 'ề': 34,
 'á': 35,
 'ể': 36,
 ' ': 37,
 'ư': 38,
 'ỳ': 39,
 'w': 40,
 'g': 41,
 't': 42,
 'ỹ': 43,
 'r': 44,
 'ĩ': 45,
 'ờ': 46,
 'ầ': 47,
 "'": 48,
 'ậ': 49,
 'ò': 50,
 'ệ': 51,
 'ỡ': 52,
 'ọ': 53,
 'ỉ': 54,
 'ạ': 55,
 'ì': 56,
 'ù': 57,
 'f': 58,
 'ắ': 59,
 'í': 60,
 'ố': 61,
 'ổ': 62,
 'ã': 63,
 'ẫ': 64,
 'ẳ': 65,
 'ô': 66,
 'ủ': 67,
 'ỏ': 68,
 'ũ': 69,
 'ử': 70,
 'l': 71,
 'â': 72,
 'è': 73,
 'ị': 74,
 'q': 75,
 'ự': 76,
 'à': 77,
 'j': 78,
 'ơ': 79,
 'ú': 80,
 'x': 81,
 'õ': 82,
 'v': 83,
 'ỷ': 84,
 's': 85,
 'b': 86,
 'ứ': 87,
 'ằ': 88,
 'p': 89,
 'đ': 90,
 'o': 91,
 'ă': 92,
 'é': 93,
 'c': 94,
 'k': 95,
 'ồ': 96}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because:

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set.

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [33]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [34]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

99

In [35]:
vocab_dict

{'ê': 0,
 'ẵ': 1,
 'n': 2,
 'ế': 3,
 'ả': 4,
 'y': 5,
 'ụ': 6,
 'ẽ': 7,
 'z': 8,
 'e': 9,
 'ẹ': 10,
 'u': 11,
 'ý': 12,
 'm': 13,
 'i': 14,
 'ẻ': 15,
 'ỗ': 16,
 'ấ': 17,
 'ẩ': 18,
 'a': 19,
 'ừ': 20,
 '<': 21,
 'ỵ': 22,
 'd': 23,
 'ễ': 24,
 'ặ': 25,
 '>': 26,
 'ộ': 27,
 'ó': 28,
 'ở': 29,
 'ữ': 30,
 'ớ': 31,
 'h': 32,
 'ợ': 33,
 'ề': 34,
 'á': 35,
 'ể': 36,
 'ư': 38,
 'ỳ': 39,
 'w': 40,
 'g': 41,
 't': 42,
 'ỹ': 43,
 'r': 44,
 'ĩ': 45,
 'ờ': 46,
 'ầ': 47,
 "'": 48,
 'ậ': 49,
 'ò': 50,
 'ệ': 51,
 'ỡ': 52,
 'ọ': 53,
 'ỉ': 54,
 'ạ': 55,
 'ì': 56,
 'ù': 57,
 'f': 58,
 'ắ': 59,
 'í': 60,
 'ố': 61,
 'ổ': 62,
 'ã': 63,
 'ẫ': 64,
 'ẳ': 65,
 'ô': 66,
 'ủ': 67,
 'ỏ': 68,
 'ũ': 69,
 'ử': 70,
 'l': 71,
 'â': 72,
 'è': 73,
 'ị': 74,
 'q': 75,
 'ự': 76,
 'à': 77,
 'j': 78,
 'ơ': 79,
 'ú': 80,
 'x': 81,
 'õ': 82,
 'v': 83,
 'ỷ': 84,
 's': 85,
 'b': 86,
 'ứ': 87,
 'ằ': 88,
 'p': 89,
 'đ': 90,
 'o': 91,
 'ă': 92,
 'é': 93,
 'c': 94,
 'k': 95,
 'ồ': 96,
 '|': 37,
 '[UNK]': 97,
 '[PAD]': 98}

In [36]:
chars_in_data = set("".join(batch_texts))
missing = chars_in_data - set(vocab_tokens)
print("Missing characters:", missing)


NameError: name 'batch_texts' is not defined

Cool, now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 30.

Let's now save the vocabulary as a json file.

In [37]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [38]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). Let's call the repo to which we will upload the files
`"wav2vec2-base-timit-demo-colab"`:

In [39]:
repo_name = "ASR-wav2vec2-challenge-SOICT-GENAI-google-colab-A100-20k-06"

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [40]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/Hung1294/ASR-wav2vec2-challenge-SOICT-GENAI-google-colab-A100-20k-06/commit/1fb6124fd796851fe5df7e00ad9219dd411ae371', commit_message='Upload tokenizer', commit_description='', oid='1fb6124fd796851fe5df7e00ad9219dd411ae371', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Hung1294/ASR-wav2vec2-challenge-SOICT-GENAI-google-colab-A100-20k-06', endpoint='https://huggingface.co', repo_type='model', repo_id='Hung1294/ASR-wav2vec2-challenge-SOICT-GENAI-google-colab-A100-20k-06'), pr_revision=None, pr_num=None)

Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-base-timit-demo-colab`

In [49]:
tokenizer.get_vocab()

{'ê': 0,
 'ẵ': 1,
 'n': 2,
 'ế': 3,
 'ả': 4,
 'y': 5,
 'ụ': 6,
 'ẽ': 7,
 'z': 8,
 'e': 9,
 'ẹ': 10,
 'u': 11,
 'ý': 12,
 'm': 13,
 'i': 14,
 'ẻ': 15,
 'ỗ': 16,
 'ấ': 17,
 'ẩ': 18,
 'a': 19,
 'ừ': 20,
 '<': 21,
 'ỵ': 22,
 'd': 23,
 'ễ': 24,
 'ặ': 25,
 '>': 26,
 'ộ': 27,
 'ó': 28,
 'ở': 29,
 'ữ': 30,
 'ớ': 31,
 'h': 32,
 'ợ': 33,
 'ề': 34,
 'á': 35,
 'ể': 36,
 'ư': 38,
 'ỳ': 39,
 'w': 40,
 'g': 41,
 't': 42,
 'ỹ': 43,
 'r': 44,
 'ĩ': 45,
 'ờ': 46,
 'ầ': 47,
 "'": 48,
 'ậ': 49,
 'ò': 50,
 'ệ': 51,
 'ỡ': 52,
 'ọ': 53,
 'ỉ': 54,
 'ạ': 55,
 'ì': 56,
 'ù': 57,
 'f': 58,
 'ắ': 59,
 'í': 60,
 'ố': 61,
 'ổ': 62,
 'ã': 63,
 'ẫ': 64,
 'ẳ': 65,
 'ô': 66,
 'ủ': 67,
 'ỏ': 68,
 'ũ': 69,
 'ử': 70,
 'l': 71,
 'â': 72,
 'è': 73,
 'ị': 74,
 'q': 75,
 'ự': 76,
 'à': 77,
 'j': 78,
 'ơ': 79,
 'ú': 80,
 'x': 81,
 'õ': 82,
 'v': 83,
 'ỷ': 84,
 's': 85,
 'b': 86,
 'ứ': 87,
 'ằ': 88,
 'p': 89,
 'đ': 90,
 'o': 91,
 'ă': 92,
 'é': 93,
 'c': 94,
 'k': 95,
 'ồ': 96,
 '|': 37,
 '[UNK]': 97,
 '[PAD]': 98,
 '<s>': 99,


### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus,
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, [Timit](hhtps://huggingface.co/datasets/timit_asr), was luckily also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining.



A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [41]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [42]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `'text'`, our datasets include two more column names `'file'` and `'audio'`. `'file'` states the absolute path of the audio file. Let's take a look.

In [43]:
timit["train"][0]["audio_file"]

'/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/vlsp2020_train_set_02/vlsp2020_train_set_02/database_sa1_Mar20_May04_cleaned_utt_0000044524-1.wav'

`Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically when calling the column `audio`. Let try it out.

In [44]:
timit["train"][0]["audio"]

{'array': [0.022125244140625,
  0.014678955078125,
  0.005706787109375,
  -0.0096435546875,
  -0.0318603515625,
  -0.05413818359375,
  -0.066497802734375,
  -0.078765869140625,
  -0.088409423828125,
  -0.10125732421875,
  -0.114349365234375,
  -0.1287841796875,
  -0.13909912109375,
  -0.151092529296875,
  -0.158660888671875,
  -0.16229248046875,
  -0.16937255859375,
  -0.176788330078125,
  -0.182403564453125,
  -0.17620849609375,
  -0.16778564453125,
  -0.165802001953125,
  -0.158294677734375,
  -0.149810791015625,
  -0.147064208984375,
  -0.136444091796875,
  -0.123077392578125,
  -0.109100341796875,
  -0.0947265625,
  -0.07733154296875,
  -0.0589599609375,
  -0.04296875,
  -0.02667236328125,
  -0.013671875,
  -0.0047607421875,
  0.0029296875,
  0.00482177734375,
  0.013092041015625,
  0.028900146484375,
  0.03985595703125,
  0.0479736328125,
  0.043304443359375,
  0.03631591796875,
  0.03485107421875,
  0.036468505859375,
  0.03350830078125,
  0.029998779296875,
  0.02789306640625,
 

We can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

The sampling rate is set to 16kHz which is what `Wav2Vec2` expects as an input.

Great, let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [45]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(timit["train"]))

print(timit["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(timit["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

lâu lâu sau này mẹ mới kể mẹ chồng em á 


It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [46]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", timit["train"][rand_int]["audio"]["sampling_rate"])

Target text: nơi người được xác nhận kể từ đầu tháng năm trong những tháng trước đó vi rút này đã có hơn một trăm ba mươi người bị nhiễm 
Input array shape: (113120,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [47]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [48]:
timit_before_encoding = timit.copy()

print("\nDataset trước khi áp dụng prepare_dataset (với encoding):")
print(timit_before_encoding)


# --- Áp dụng map với hàm prepare_dataset đã sửa ---
# Chỉ loại bỏ các cột không cần thiết khác, GIỮ LẠI cột 'text'
columns_to_remove = ['audio', 'audio_file', 'transcript_file', 'input_length'] # Liệt kê các cột muốn loại bỏ
# Đảm bảo 'text' không có trong list này nếu bạn muốn giữ lại nó

# Kiểm tra xem các cột cần loại bỏ có tồn tại trong dataset hay không
# Điều này tránh lỗi nếu bạn đã chạy prepare_dataset trước đó và các cột đã bị loại bỏ
existing_columns_to_remove = [col for col in columns_to_remove if col in timit["train"].column_names]

timit = timit.map(
    prepare_dataset,
    remove_columns=existing_columns_to_remove, # Chỉ loại bỏ các cột tồn tại
    num_proc=4
)

print("\nDataset sau khi áp dụng prepare_dataset (labels đã được thêm, các cột không cần thiết đã bị loại bỏ, cột 'text' được giữ lại):")
print(timit)


Dataset trước khi áp dụng prepare_dataset (với encoding):
{'train': Dataset({
    features: ['audio_file', 'transcript_file', 'text', 'audio'],
    num_rows: 18183
}), 'test': Dataset({
    features: ['audio_file', 'transcript_file', 'text', 'audio'],
    num_rows: 2021
})}


Map (num_proc=4):   0%|          | 0/18183 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same cal

Map (num_proc=4):   0%|          | 0/2021 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
/usr/local/lib/python3.11/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same cal


Dataset sau khi áp dụng prepare_dataset (labels đã được thêm, các cột không cần thiết đã bị loại bỏ, cột 'text' được giữ lại):
DatasetDict({
    train: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 2021
    })
})


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.


In [50]:
def check_data_preparation(dataset, processor, num_examples=5):
    print(f"Kiểm tra tiền xử lý trên {num_examples} mẫu ngẫu nhiên từ tập dữ liệu:")

    for _ in range(num_examples):
        random_index = random.randint(0, len(dataset) - 1)
        sample = dataset[random_index]

        encoded_labels = sample["labels"]
        decoded_text = processor.batch_decode([encoded_labels], group_tokens=False)[0]

        # Lấy văn bản gốc từ cột 'text' (đã được giữ lại)
        original_text = sample["text"]

        print(f"\nMẫu index: {random_index}")
        # print(f"Encoded labels: {encoded_labels}") # Có thể bỏ comment nếu muốn xem labels id
        print(f"Decoded text (from labels): '{decoded_text}'")
        print(f"Original text (after cleaning): '{original_text}'")

        # So sánh và in kết quả
        if decoded_text.strip() == original_text.strip().replace(" ", "|"): # So sánh, có thể cần điều chỉnh cách so sánh nếu xử lý khoảng trắng khác
             print("Kết quả giải mã khớp với văn bản gốc (đã xử lý ký tự đặc biệt).")
        else:
             print("Kết quả giải mã KHÔNG khớp với văn bản gốc (đã xử lý ký tự đặc biệt).")
             # In chi tiết hơn để debug
             print(f"Decoded length: {len(decoded_text)}")
             print(f"Original length: {len(original_text)}")
             # Có thể thêm logic để so sánh từng ký tự hoặc sử dụng difflib để tìm sự khác biệt


# Chạy hàm kiểm tra với dataset đã cập nhật
check_data_preparation(timit["train"], processor, num_examples=10)

Kiểm tra tiền xử lý trên 10 mẫu ngẫu nhiên từ tập dữ liệu:

Mẫu index: 1821
Decoded text (from labels): 'liên quan đến vụ sản phụ tại bình phước bị tài xế bỏ rơi giữa đường trong lúc đi sinh gây bức xúc dư luận trong thời gian vừa qua hôm nay công an huyện bù đăng cùng viện kiểm sát nhân dân huyện bù đăng đã đến hiện trường để điều tra vụ việc'
Original text (after cleaning): 'liên quan đến vụ sản phụ tại bình phước bị tài xế bỏ rơi giữa đường trong lúc đi sinh gây bức xúc dư luận trong thời gian vừa qua hôm nay công an huyện bù đăng cùng viện kiểm sát nhân dân huyện bù đăng đã đến hiện trường để điều tra vụ việc '
Kết quả giải mã KHÔNG khớp với văn bản gốc (đã xử lý ký tự đặc biệt).
Decoded length: 240
Original length: 241

Mẫu index: 13387
Decoded text (from labels): 'cảm ơn đời đã cho ta từng sớm mai nụ cười là tia nắng gieo vui trong'
Original text (after cleaning): 'cảm ơn đời đã cho ta từng sớm mai nụ cười là tia nắng gieo vui trong '
Kết quả giải mã KHÔNG khớp với văn bản gốc (đ

In [51]:
timit["train"][0]
timit

DatasetDict({
    train: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 2021
    })
})

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset.

In [ ]:
# max_input_length_in_sec = 4.0
# timit["train"] = timit["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Filter:   0%|          | 0/18183 [00:00<?, ? examples/s]

In [52]:
timit["train"]

Dataset({
    features: ['text', 'input_values', 'input_length', 'labels'],
    num_rows: 18183
})

Awesome, now we are ready to start training!

In [53]:
# prompt: lưu dataset lên driver

# ... (các bước tiền xử lý dataset của bạn ở trên) ...

print("Dataset sau khi tách:")
print(timit)

# --- Kết thúc phần code tiền xử lý dataset của bạn ---

# Tạo thư mục lưu nếu chưa có
os.makedirs(processed_dataset_path, exist_ok=True)

# Lưu dataset train và test sau khi tiền xử lý xong
timit["train"].save_to_disk(os.path.join(processed_dataset_path, "train"))
timit["test"].save_to_disk(os.path.join(processed_dataset_path, "test"))

print(f"Dataset đã được tiền xử lý và lưu tại: {processed_dataset_path}")



Dataset sau khi tách:
DatasetDict({
    train: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['text', 'input_values', 'input_length', 'labels'],
        num_rows: 2021
    })
})


Saving the dataset (0/16 shards):   0%|          | 0/18183 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/2021 [00:00<?, ? examples/s]

Dataset đã được tiền xử lý và lưu tại: /content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k


In [3]:
# prompt: import drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
from datasets import load_dataset, load_from_disk, DatasetDict

In [5]:
# prompt: code load data từ driver /content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k

# Giữ nguyên các import khác từ notebook của bạn
#Đường dẫn để lưu và tải dataset trên Google Drive
processed_dataset_path = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k"

# Mount Google Drive
# drive.mount('/content/drive') # Assuming this is already mounted from the previous code

# Thay thế bằng đường dẫn thực tế của bạn
data_dir = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k"

# Kiểm tra xem dataset đã tồn tại trên Drive chưa
if os.path.exists(os.path.join(processed_dataset_path, "train")) and \
   os.path.exists(os.path.join(processed_dataset_path, "test")):
    print(f"Dataset đã tồn tại tại {processed_dataset_path}. Đang tải từ Drive...")
    # Tải dataset train và test
    loaded_train_dataset = load_from_disk(os.path.join(processed_dataset_path, "train"))
    loaded_test_dataset = load_from_disk(os.path.join(processed_dataset_path, "test"))

    # Gán lại cho biến timit để sử dụng
    timit = DatasetDict({
        "train": loaded_train_dataset,
        "test": loaded_test_dataset
    })
    print("Dataset đã được tải thành công từ Drive:")
    print(timit)

else:
    print(f"Dataset chưa tồn tại tại {processed_dataset_path}. Vui lòng chạy lại phần tiền xử lý dataset trước.")



Dataset đã tồn tại tại /content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/processed_dataset_20k. Đang tải từ Drive...


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Dataset đã được tải thành công từ Drive:
DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 18183
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 2021
    })
})


In [7]:
# prompt:  lưu dataset đầy đủ vào 1 biến khác,  lấy 1/2 số mẫu trong dataset  lưu vào timit

# Lưu dataset đầy đủ vào một biến khác
full_dataset = timit.copy()

# Lấy 1/2 số mẫu từ tập huấn luyện và lưu vào timit
train_size = len(full_dataset["train"])
half_train_size = train_size // 2
random_indices = random.sample(range(train_size), half_train_size)

timit["train"] = full_dataset["train"].select(random_indices)

print("Dataset gốc (full_dataset):")
print(full_dataset)

print("\nDataset sau khi chỉ giữ lại 1/2 tập train (timit):")
timit


Dataset gốc (full_dataset):
{'train': Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 18183
}), 'test': Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 2021
})}

Dataset sau khi chỉ giữ lại 1/2 tập train (timit):


DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 9091
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 2021
    })
})

In [26]:
# prompt: gán lại full_dataset cho timit

# Gán lại full_dataset cho timit
timit = full_dataset.copy()

print("\nDataset sau khi gán lại full_dataset cho timit:")
timit


Dataset sau khi gán lại full_dataset cho timit:


{'train': Dataset({
     features: ['input_values', 'input_length', 'labels'],
     num_rows: 18183
 }),
 'test': Dataset({
     features: ['input_values', 'input_length', 'labels'],
     num_rows: 2021
 })}

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [54]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [55]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [56]:
wer_metric = load_metric("wer")

<ipython-input-56-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [57]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [58]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=tokenizer.vocab_size
)

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  if self.torch_dtype is not None and isinstance(self.torch_dtype, str):


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [22]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [62]:
from transformers import TrainingArguments
output_dir = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/ASR-model-20k-04"

training_args = TrainingArguments(
  output_dir=output_dir,
  group_by_length=True,
  per_device_train_batch_size=16,
  eval_strategy="steps",
  num_train_epochs=12,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=5e-5,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [63]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

<ipython-input-63-00e26c1cf103>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 270 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 16 or even less and eventually make use of [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [ ]:
trainer.train()

The final WER should be around 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1747491425.e0289bbb38cb.725.0:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

events.out.tfevents.1747475958.2c48f0e0adc8.2538.0:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Hung1294/ASR-model-20k-02/commit/bd5f2abd5c3165ac302a28a7faae3b81b895feef', commit_message='End of training', commit_description='', oid='bd5f2abd5c3165ac302a28a7faae3b81b895feef', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Hung1294/ASR-model-20k-02', endpoint='https://huggingface.co', repo_type='model', repo_id='Hung1294/ASR-model-20k-02'), pr_revision=None, pr_num=None)

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

In [ ]:
# processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")

In [ ]:
# model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Đường dẫn đến checkpoint cuối cùng đã lưu trên Drive
checkpoint_path = "/content/drive/MyDrive/AI_SOICT/20242/Audio_Generation_IT5424_A/Challenge ASR/output_dir/checkpoint-7620" # Thay xxxxx bằng số checkpoint

model = Wav2Vec2ForCTC.from_pretrained(checkpoint_path).cuda()

# Processor nên được tạo ra từ vocab.json và cấu hình feature extractor của bạn
# Hoặc tải từ đường dẫn bạn đã push lên Hub nếu bạn đã làm
vocab_path = "./vocab.json" # Hoặc đường dẫn đến vocab.json trên Drive nếu bạn đã lưu
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
tokenizer = Wav2Vec2CTCTokenizer(vocab_path, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`.

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [ ]:
results = timit["test"].map(map_to_result, remove_columns=timit["test"].column_names)

Parameter 'function'=<function map_to_result at 0x7e19fea48540> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/2021 [00:00<?, ? examples/s]

Let's compute the overall WER now.

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 1.000


21.8% WER - not bad! Our demo model would have probably made it on the official [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit).

Let's take a look at some predictions to see what errors are made by the model.

In [ ]:
show_random_elements(results)

,pred_str,text
0,yỹõắ yõyởõắhìắbõũâắõãvỷõêỉõyâắbõẩễõắấáõ,mĩ nam mĩ nữ giãn hết cơ mặt xem nào
1,úấõãdãõêẹõêàõểvcộyõvcựắõưvốõêủắbõắvcõơễắõãệưõêó õêjêvõ õắaưõữàõêiắõõofõbẩdãõvẹcõúửyõýzõẩễyõúgyõ,là tất cả các vị nguyên thủ cũng như dân tộc của các a nước đó cũng sẽ rất hài lòng vì tấm lòng
2,ôờõviắbõ>õýdỷõêjcõưváắbõbá ẹcõúấõưvửắbõãvhụắbõvjắõữcõãvxõvẽyõvợắõểvámõõêvcõýễỷõbcỳõũcõơpõắvh õúấõòcõơxõắồắõắbhỳcõã cõýcaãõắvồắbõắb cvõyấõêó õắáấõữồ ắõãáấõíòcõơcấõêvằõưvửắbõblcõúấõưvửắõvâắõýĩãõêícõểêjêvõôỗắõữộõểvímõbxõyấõểvámõ,sao không ở mấy cái phòng gọi là phòng thượng hạng đi thì hẵng khoe chứ bây giờ ví dụ như là ôi giồi nếu người ta biết những ai mà cũng ở đấy bảo ui giồi phòng đấy là phòng hạng bét của khách sạn lấy khoe gì mà khoe
3,ãẩáắbõyíỵêõúợêõêvếõiắbõêàõêỉõvlcõũẫõãvdyõêv õãâcõícaãõắ yõêv õêó õgyõữâỷõbơịõợyõúâcõũấõýờắõắvcỡưõgắbõúấõểaãõắ yõắv õắỵcõữiíõêvzõvhỉcõôằõ,trong một lần khi ông có cơ hội về thăm cha tại kinh nam cha của ông đã giữ ông lại và bổ nhiệm ông là kinh nam nha nội đô chỉ huy sứ
4,ưvcỡãõữaắõữỵắbõãẩợêõbcáõữàõêủắõýzõý>õbcụõêjêõ,việc đánh đổng trác do đó cũng bị bỏ dở các <unk>
5,íắểsãẹyõêíỷõvâêõýcõýờắbõữhụõạíờêõểvờcõểvẹâõýởõýờắõêcõvệêõêrõôjíõãviắbõạí õắb cõýccõêvzắõãvâyõyỉcõýỵãõắgyõy ỷõểiắõýzõãdyõ,<unk> tám quy hắt mười bốn được quốc hội khóa mươi bốn kỳ họp thứ sáu thông qua ngày mười chín tháng mười một năm hai không mười tám
6,ãcaắvõắvcễắõêvóõãzắvõzõý ắõắvhõơễắõãịắõưvjyõắbệưõãvlõêốắbõữcẫíõởõúấõơíõữấõắpõúhụêõắvhắbõãvựắõãẩ ắbõã cõắấắbõbc õểviắbõãẩựắõữz õýấắõãịắõũặắõêvồỡắbõữcỡắbõẩdãõũhằêõãâưõữễỷõêốắbõúấcõũỗắõữvộõêặắõưvẹcõãẩgắbõãẩụõoíỷắõắbvzõ,tuy nhiên chủ tịch ủy ban nhân dân tỉnh phan ngọc thọ cũng lưu ý là dù đã nỗ lực nhưng tình trạng tai nạn giao thông trên địa bàn tỉnh vẫn chuyển biến rất phức tạp đây cũng là vấn đề cần phải trăn trở suy nghĩ
7,êủắbõắvcựõơốắbõêjêõữâcõãvíờêõắvhõ õúiõãcựíõẩxcõắàắõãvờõêíờêõêvaõãiắbõểvpưõơíõẩcỡõ õotắõũấõbơcíõẩễỷõáêõôfõữễỷõữộõbc yõắiắbõữỵõơcíõẩcỡêõ õoõ,cũng như dùng các loại thuốc như allopurinol và urate oxidase để giảm nồng độ uric acid
8,víắbõãẩxắvõũìcõôịõãâcõãẩụõêvếắvõêó õắvẹắbõvấắbõãvĩưõũcõắ õểcõiõễỷõãvĩưõôỉõơcịắõắvỗãõýjắõãcắõãvặắõãvĩưõ,chương trình với sự tài trợ chính của nhãn hàng thép vi na ki ô uây thép xây dựng nhật bản tinh thần thép
9,êvíắbõã ỷõvỵõãẩỳõofõbxắbõắvấrõãváõắbhỳcõêệõêiắbõêàõưvgắbõêẹắõữẽưõýc õểvlõểvgắbõãẩựắõữzõýấắõãáấắõãxắvõ,chung tay hỗ trợ xây dựng nhà ở cho người có công có hoàn cảnh đặc biệt khó khăn trên địa bàn toàn tỉnh


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

Finally, to better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens.

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ê ằ [PAD] [PAD] [PAD] [PAD] [PAD] õ õ [PAD] ã ã v v v ĩ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] õ õ [PAD] [PAD] ý ẹ [PAD] [PAD] [PAD] á á á õ õ õ [PAD] [PAD] ô | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] õ õ [PAD] [PAD] ê á [PAD] [PAD] [PAD] [PAD] ắ ắ õ õ [PAD] [PAD] ẩ ẩ | [PAD] [PAD] [PAD] [PAD] [PAD] c c õ õ ú â [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] c õ õ [PAD] ữ ữ ộ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] í í [PAD] [PAD] õ õ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ý â [PAD] [PAD] [PAD] ắ ắ b õ õ [PAD] ư ư v h ỉ [PAD] ắ b b õ õ ư v v h ỉ [PAD] [PAD] [PAD] b õ õ ắ v v â [PAD] [PAD] ắ ắ õ õ õ [PAD] [PAD] o ĩ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] ê ê õ õ [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

The output should make it a bit clearer how CTC works in practice. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.

I again advise the reader to take a look at [this](https://distill.pub/2017/ctc) very nice blog post to better understand CTC.